In [3]:
import read__geolife as rg
import os, time
import pandas as pd

In [4]:
os.getcwd()

'/home/lizhenyu/Project/E2DTC/preprocess'

In [5]:
rg.read_all_users()

TypeError: read_all_users() missing 1 required positional argument: 'folder'

In [6]:
def del_traj_day(df, col_name):
    d = pd.DataFrame(df[col_name].value_counts())
    d.columns = ['nums']
    # 少于5天的次的全部删除
    d = d[d['nums'] < 5]
    delindexs = d.index
    print(len(delindexs))
    return df[~df[col_name].isin(delindexs)]

In [13]:
def get_ts_mode_mean(df):
    t1 = df.groupby(['lat', 'lon']).size().reset_index()
    t1.columns = ['a', 'b', 'count']
    t2 = t1.groupby(['a'])['count'].max().reset_index()
    t2.columns = ['a', 'max_count']
    t1 = t1.merge(t2, on=['a'], how='left')
    t1 = t1[t1['count']==t1['max_count']]
    t1 = t1.groupby(['a'])['b'].mean().reset_index()
    t1

In [14]:
# 每個timeslot的軌跡是最後一次出現的軌跡
def get_slot_by_unix(ts):
    dt = time.localtime(ts)
    day, hour, minute = dt.tm_mday, dt.tm_hour, dt.tm_min
    slot = hour * 2 + (0 if minute <= 30 else 1)
    return slot

In [15]:
def ts_to_slot(ts):
    ans = [0] * 48
    for t in ts:
        slot = get_slot_by_unix(t)
        ans[slot] = 1
    return ans

In [16]:
def traj_to_slot(trajectory, ts, pad=0):
    ans = [pad] * 48
    for i in range(len(ts)):
        slot_id = get_slot_by_unix(ts[i])
        ans[slot_id] = trajectory[i]
    return ans

In [17]:
def get_loc_index(lon, lat):
    df = pd.read_csv('./loc_index.csv')


In [9]:
# all users
df_all = rg.read_all_users('../data/geolife/Data')
df_all.sort_values(by=['time'],inplace=True)

NameError: name 'rg' is not defined

In [21]:
df_all
df_all.to_hdf('./Geolife_all_user.h5',key='data')

In [92]:
# 删除非北京的采样点
lon = [115.416827, 117.508251]
lat = [39.442078, 41.058964]
df = df_all[(df_all.lat>lat[0])&(df_all.lat<lat[1])&(df_all.lon>lon[0]) & (df_all.lon<lon[1])]
df

,time,lat,lon,alt,label,user
0,2000-01-01 23:12:19,39.988992,116.327023,128.937005,0,163
1,2000-01-01 23:13:21,39.990964,116.327041,221.128615,0,163
2,2000-01-01 23:15:23,39.993207,116.326827,217.191591,0,163
0,2007-04-12 09:31:32,39.974233,116.330383,823.490814,0,142
1,2007-04-12 09:39:37,39.974317,116.330450,823.490814,0,142
...,...,...,...,...,...,...
291,2012-07-27 08:31:03,39.979481,116.303499,131.760207,0,163
292,2012-07-27 08:31:05,39.979475,116.303497,131.097241,0,163
293,2012-07-27 08:31:10,39.979374,116.303522,126.235679,0,163
294,2012-07-27 08:31:15,39.979190,116.303587,119.301427,0,163


In [96]:
df['timestamp'] = df['time'].apply(lambda x: int(x.timestamp()))
df['year'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_year)
df['month'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mon)
df['day'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mday)
df['slot_id'] = df['timestamp'].apply(lambda x: get_slot_by_unix(x))
df = df.drop_duplicates(subset=['user', 'year', 'month', 'day', 'slot_id'])
df.sort_values(by='user',inplace=True)
df.reset_index(drop=True, inplace=True)
df

/tmp/ipykernel_2214526/3021142272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = df['time'].apply(lambda x: int(x.timestamp()))
/tmp/ipykernel_2214526/3021142272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_year)
/tmp/ipykernel_2214526/3021142272.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,time,lat,lon,alt,label,user,timestamp,year,month,day,slot_id
0,2009-05-12 05:31:02,40.008054,116.324130,159.000000,0,0,1242106262,2009,5,12,27
1,2009-04-14 04:43:25,39.762913,116.295306,492.000000,0,0,1239684205,2009,4,14,25
2,2009-06-16 14:58:00,40.001417,116.321702,113.000000,0,0,1245164280,2009,6,16,45
3,2009-06-16 15:00:00,39.999736,116.324415,51.000000,0,0,1245164400,2009,6,16,46
4,2009-06-16 16:16:38,39.999935,116.327138,106.000000,0,0,1245168998,2009,6,17,0
...,...,...,...,...,...,...,...,...,...,...,...
72401,2007-12-11 05:49:32,39.922650,116.348017,147.637795,0,181,1197352172,2007,12,11,27
72402,2007-12-11 05:00:50,39.986733,116.298917,154.199475,0,181,1197349250,2007,12,11,26
72403,2007-12-11 04:47:55,39.982233,116.301467,259.186352,0,181,1197348475,2007,12,11,25
72404,2007-12-10 05:18:40,39.984433,116.299933,127.952756,0,181,1197263920,2007,12,10,26


In [97]:
lon_list = (df['lon'].value_counts().index.tolist())
lon_list.sort(reverse=True)
lon_list

[117.508232,
 117.50823,
 117.5082283,
 117.5081933,
 117.508193,
 117.5081799,
 117.5081483,
 117.508128,
 117.508117,
 117.508068,
 117.508019,
 117.5079616,
 117.5079416,
 117.507684,
 117.50767,
 117.50749,
 117.507419,
 117.507126,
 117.5071199,
 117.506954,
 117.5068066,
 117.5067283,
 117.5062233,
 117.5058766,
 117.5054216,
 117.5051616,
 117.5047,
 117.504425,
 117.50165,
 117.4988049,
 117.4944433,
 117.485905,
 117.4856483,
 117.48541,
 117.484366666667,
 117.4828,
 117.48245,
 117.48209,
 117.481766666667,
 117.4816,
 117.481366666667,
 117.480433333333,
 117.479533333333,
 117.47795,
 117.476246,
 117.473516666667,
 117.4712,
 117.4701833,
 117.467933333333,
 117.466133333333,
 117.464466666667,
 117.4631883,
 117.4619899,
 117.460535,
 117.458733333333,
 117.458183333333,
 117.4575216,
 117.456816666667,
 117.455166666667,
 117.45045,
 117.449143,
 117.44744,
 117.4472,
 117.437633333333,
 117.437133333333,
 117.431445,
 117.42975,
 117.4288399,
 117.423155,
 117.41605,
 

df['timestamp'] = df['time'].apply(lambda x: int(x.timestamp()))
df['year'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_year)
df['month'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mon)
df['day'] = df['timestamp'].apply(lambda x: time.localtime(x).tm_mday)
df['slot_id'] = df['timestamp'].apply(lambda x: get_slot_by_unix(x))
df = df.drop_duplicates(subset=['user', 'year', 'month', 'day', 'slot_id'])
df.sort_values(by='user',inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [102]:
loc_index = df[['lon','lat']]
loc_index = loc_index.sort_values(by='lon').drop_duplicates().reset_index(drop=True)
loc_index.to_csv('loc_index.csv')

In [103]:
loc_index

,lon,lat
0,115.416875,40.444661
1,115.418200,40.445192
2,115.481076,39.981526
3,115.495228,40.021426
4,115.496418,40.030745
...,...,...
68584,117.508193,41.003427
68585,117.508193,40.443502
68586,117.508228,39.727523
68587,117.508230,39.945472


In [105]:
loc_index_dict

{(115.416875, 40.444661): 0,
 (115.4181999, 40.4451916): 1,
 (115.481076, 39.981526): 2,
 (115.495228, 40.021426): 3,
 (115.496418, 40.030745): 4,
 (115.496996, 40.030881): 5,
 (115.499326, 40.029518): 6,
 (115.500765, 40.02847): 7,
 (115.503269, 40.025695): 8,
 (115.509127, 40.024848): 9,
 (115.51088, 40.020323): 10,
 (115.512377, 40.40614): 11,
 (115.51271, 40.888247): 12,
 (115.51339, 40.021242): 13,
 (115.5210416, 40.4277633): 14,
 (115.531725, 40.884182): 15,
 (115.543735, 39.922438): 16,
 (115.5551166, 39.8616233): 17,
 (115.5725366, 39.8382283): 18,
 (115.5744816, 39.8363983): 19,
 (115.5773633, 39.8352516): 20,
 (115.577785, 39.807985): 21,
 (115.581133, 39.812474): 22,
 (115.5820966, 39.6574483): 23,
 (115.5823733, 39.8314499): 24,
 (115.5824583, 39.83088): 25,
 (115.5832666, 39.8304183): 26,
 (115.585676, 39.635858): 27,
 (115.5868766, 39.8287816): 28,
 (115.5925333, 39.8225766): 29,
 (115.5944949, 39.8248749): 30,
 (115.5948416, 39.8247866): 31,
 (115.5973016, 39.8276416): 3

In [106]:
# dict:(lat,lon)--->index
loc = [tuple(x) for x in loc_index.values.tolist()]
index = loc_index.index.tolist()
loc_index_dict = dict(zip(loc, index))

tmp = df[['lon', 'lat']]
index = []
for i in range(len(tmp)):
    index.append(loc_index_dict[(tmp.iloc[i][0], tmp.iloc[i][1])])
df['loc_index'] = index
df

/tmp/ipykernel_2214526/1084782651.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc_index'] = index


,time,lat,lon,alt,label,user,timestamp,year,month,day,slot_id,loc_index
0,2009-05-12 05:31:02,40.008054,116.324130,159.000000,0,0,1242106262,2009,5,12,27,21953
1,2009-04-14 04:43:25,39.762913,116.295306,492.000000,0,0,1239684205,2009,4,14,25,5988
2,2009-06-16 14:58:00,40.001417,116.321702,113.000000,0,0,1245164280,2009,6,16,45,20143
3,2009-06-16 15:00:00,39.999736,116.324415,51.000000,0,0,1245164400,2009,6,16,46,22185
4,2009-06-16 16:16:38,39.999935,116.327138,106.000000,0,0,1245168998,2009,6,17,0,25505
...,...,...,...,...,...,...,...,...,...,...,...,...
72401,2007-12-11 05:49:32,39.922650,116.348017,147.637795,0,181,1197352172,2007,12,11,27,41922
72402,2007-12-11 05:00:50,39.986733,116.298917,154.199475,0,181,1197349250,2007,12,11,26,6777
72403,2007-12-11 04:47:55,39.982233,116.301467,259.186352,0,181,1197348475,2007,12,11,25,8123
72404,2007-12-10 05:18:40,39.984433,116.299933,127.952756,0,181,1197263920,2007,12,10,26,7155


In [107]:
df['loc_index'].max()

68588

In [108]:
seqs = []
for (user_index, year, month, day), group in df.groupby(['user', 'year', 'month', 'day']):
    group.reset_index(inplace=True)
    if group.shape[0] < 12:
        continue
    ts = group['timestamp']
    seq = group['loc_index'].tolist()
    seq = traj_to_slot(seq, ts, pad='[PAD]')

    if seq.count('[PAD]') > 36:
        continue
    seq = [str(x) for x in seq]
    seqs.append([" ".join(seq), user_index, year, month, day])

    # print('uid:', user_index,'date:', month, '-', day)
    # print("group shape:", group.shape, '[PAD] count:', seq.count('[PAD]'))
    # print(group.head())

    # print(group)
    # print(ts)
    # print(seq)
    # print(seqs)

In [109]:
all_traj = pd.DataFrame(seqs, columns=['trajectory', 'user_index', 'year', 'month', 'day'])
all_traj = del_traj_day(all_traj,'user_index')
all_traj

56


,trajectory,user_index,year,month,day
0,22042 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3
2,15374 15429 15665 15466 15874 15502 15653 1554...,0,2009,4,4
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6
...,...,...,...,...,...
1833,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13
1834,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15
1835,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2
1836,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12


In [110]:
all_traj.reset_index(drop= True)

,trajectory,user_index,year,month,day
0,22042 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,3,31
1,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,3
2,15374 15429 15665 15466 15874 15502 15653 1554...,0,2009,4,4
3,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,5
4,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,0,2009,4,6
...,...,...,...,...,...
1726,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,13
1727,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,9,15
1728,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,2
1729,[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,179,2008,10,12


In [111]:
all_traj.to_hdf('all_traj.h5', key='data')

In [112]:
# num users
len(dict(all_traj['user_index'].value_counts()))

66

In [320]:
# duration
print(all_traj['year'].max(), all_traj['year'].min())

2012 2007
